In [1]:
import pandas as pd

##### researcher

In [2]:
PeoInfo = pd.read_csv('researcher.csv')

In [3]:
PeoInfo.head(5)

,PID,FirstName,MiddleName,LastName,Institution,InstitutionMAGID,ResearchArea,ORCID,MAGAuthorID
0,1,STEPHEN,V,DAVID,Oregon Health and Science University,165690674.0,"neuro,csd,bme",0000-0003-4135-3104,2.171828e+09
1,2,BENJAMIN,Y,HAYDEN,"University of Minnesota, Twin Cities",130238516.0,neuro,0000-0002-7678-4281,2.082173e+09
2,3,JACK,L,GALLANT,"University of California, Berkeley",95457486.0,"neuro,psych",NaN,2.110166e+09
3,4,BENJAMIN,NaN,WILLMORE,University of Oxford,40120149.0,neuro,NaN,2.183264e+09
4,5,KENDRICK,NORRIS,KAY,"University of Minnesota, Twin Cities",130238516.0,neuro,NaN,2.156587e+09


In [4]:
PeoInfo = PeoInfo[['PID','MAGAuthorID']][~PeoInfo.MAGAuthorID.isnull()]

In [5]:
magId = [int(k) for k in list(PeoInfo.MAGAuthorID)]

In [6]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

In [7]:
magId_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = magId[start_point:end_point]
        magId_groups.append(temp)
    except:
        temp = magId[start_point:]
        magId_groups.append(temp)
        
    start_point = end_point
    if end_point>len(magId):
        break

In [8]:
print(len(magId_groups))

99


In [9]:
l = len(magId_groups)
for i in range(l):
    temp =  magId_groups[i]
    sql_query = """select authors_ids.author_id, authors_ids.mag 

                    from authors_ids 

                    where authors_ids.mag in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))

data = tp

In [10]:
data = data.rename(columns={'mag':'MAGAuthorID'})

In [11]:
data

,author_id,MAGAuthorID
0,A367921,367921
1,A2214144,2214144
2,A2626456,2626456
3,A3881444,3881444
4,A3966264,3966264
...,...,...
4199,A3085024380,3085024380
4200,A3086004369,3086004369
4201,A3087060357,3087060357
4202,A3087666585,3087666585


In [12]:
PeoInfo = PeoInfo.merge(data,on='MAGAuthorID',how='left')

In [13]:
PeoInfo.head(5)

,PID,MAGAuthorID,author_id
0,1,2.171828e+09,A2171827615
1,2,2.082173e+09,A2082172561
2,3,2.110166e+09,A2110165986
3,4,2.183264e+09,A2183263527
4,5,2.156587e+09,A2156586888


In [14]:
PeoInfo.to_csv('pid_mag_openalex.csv',index=0)